<a href="https://colab.research.google.com/github/arpdm/predictive-maintenance-platform/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Flags
COLAB_ACTIVE = True
GENERATE_PLOTS = False

# Dataset
if COLAB_ACTIVE:
    BASE_PATH = "/content/drive/MyDrive/PdM/data_set/"
    DATA_PATH = "/content/drive/MyDrive/PdM/model_generated_data"
else:
    BASE_PATH = "/data_set/"
    DATA_PATH = "/model_generated_data/"

DS_001 = BASE_PATH + "N-CMAPSS_DS01-005.h5"
DS_002 = BASE_PATH + "N-CMAPSS_DS02-006.h5"
DS_003 = BASE_PATH + "N-CMAPSS_DS03-012.h5"
DS_004 = BASE_PATH + "N-CMAPSS_DS04.h5"
DS_005 = BASE_PATH + "N-CMAPSS_DS05.h5"
DS_006 = BASE_PATH + "N-CMAPSS_DS06.h5"
DS_007 = BASE_PATH + "N-CMAPSS_DS07.h5"
DS_008 = BASE_PATH + "N-CMAPSS_DS08a-009.h5"
DS_009 = BASE_PATH + "N-CMAPSS_DS08c-008.h5"
DS_010 = BASE_PATH + "N-CMAPSS_DS08d-010.h5"

# Model Parameters
HIST_WINDOW = 30
MINI_BATCH_SIZE = 256
EPOCHS = 40
STEPS = 100
PATIENCE_EPOCH = 10
LEARNING_RATE = 6e-4
FIRST_LAYER_UNITS = 50
SECOND_LAYER_UNITS = 100
DROPOUT = 0.2
BETA_1 = 0.9  # Momentum
BETA_2 = 0.999
EPSILON = 1e-8
CYCLE_L1 = 15
CYCLE_L2 = 5



---
# COLAB ONLY
---

In [2]:
from google.colab import drive

drive.mount("/content/drive/")

!cp drive/MyDrive/PdM/predictive-maintenance-platform/engine_data.py .
!cp drive/MyDrive/PdM/predictive-maintenance-platform/plot_util.py .
!cp drive/MyDrive/PdM/predictive-maintenance-platform/mark_001_model.py .
!cp drive/MyDrive/PdM/predictive-maintenance-platform/pcoe_engine_data_visualizer.py .

Mounted at /content/drive/


In [ ]:
pip install tensorflow-addons

---
# END OF COLAB ONLY
---

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time

from engine_data import EngineData
from pcoe_engine_data_visualizer import PcoeEngingeVis
from plot_util import PlotUtil
from mark_001_model import Mark001Model

In [ ]:
# Load data set and prepare data frames
ed = EngineData()
pcoe_enging_vis = PcoeEngingeVis(ed)
plot_util = PlotUtil()

In [ ]:
mark_001 = Mark001Model()

In [7]:
# Dataset load block
ed.load_hdf5_to_numpy_arr(DS_001)

In [8]:
ed.df_x_s_train = pd.concat([ed.df_x_s_train, ed.df_x_v_train], axis=1)
ed.df_x_s_train = pd.concat([ed.df_x_s_train, ed.df_w_train], axis=1)
ed.df_x_s_train = pd.concat([ed.df_x_s_train, ed.df_t_train], axis=1)

ed.df_x_s_test = pd.concat([ed.df_x_s_test, ed.df_x_v_test], axis=1)
ed.df_x_s_test = pd.concat([ed.df_x_s_test, ed.df_w_test], axis=1)
ed.df_x_s_test = pd.concat([ed.df_x_s_test, ed.df_t_test], axis=1)

ed.df_x_s_train = ed.add_labels_to_dataset(ed.df_x_s_train, failure_in_w_cycles_1=CYCLE_L1, failure_in_w_cycles_2=CYCLE_L2)
ed.df_x_s_train = ed.normalize_dataset(ed.df_x_s_train)
ed.df_x_s_test = ed.add_labels_to_dataset(ed.df_x_s_test, failure_in_w_cycles_1=CYCLE_L1, failure_in_w_cycles_2=CYCLE_L2)
ed.df_x_s_test = ed.normalize_dataset(ed.df_x_s_test)

In [ ]:
df_x_s_u_c = ed.df_x_s_train.loc[(ed.df_aux_train.unit == 1)]
ax1 = df_x_s_u_c.plot(subplots=True, sharex=True, figsize=(20,20))

In [9]:
def train():

    # Hyperparameters required for ADAM Optimizer
    adam_params = {
        "learning_rate": LEARNING_RATE,
        "beta_1": BETA_1,
        "beta_2": BETA_2,
        "epsilon": EPSILON,
    }

    features = ed.df_x_s_train.columns.drop(["RUL", "label1", "label2", "cycle", "id"]).size
    model_mark_001 = mark_001.create_binary_classifier_model(
        HIST_WINDOW,
        features,
        units_first_layer=FIRST_LAYER_UNITS,
        units_second_layer=SECOND_LAYER_UNITS,
        dropout_rate=DROPOUT,
        optimizer_hyperparams=adam_params,
        adam_w_enabled=False,
    )

    # We want to train for all engines in the training dataset
    for engine_unit in np.unique(ed.df_aux_train["unit"]):

        single_engine_train = ed.generate_data_frame_for_specific_engine(ed.df_x_s_train, engine_unit)
        x_train, y_train = ed.generate_x_y_model_inputs(single_engine_train, window=HIST_WINDOW)
        mark_001_history = mark_001.fit_model(x_train, y_train, EPOCHS, STEPS, MINI_BATCH_SIZE, PATIENCE_EPOCH)
        mark_001.save_model(mark_001_history, DATA_PATH, "Mark_001_Pickle")

    if GENERATE_PLOTS:
        plot_util.generate_training_history_accuracy_and_loss_plot(mark_001_history, DATA_PATH)
    return mark_001_history

In [10]:
def test():
    for engine_unit in np.unique(ed.df_aux_test["unit"]):

        print("========================================================================================")

        single_engine_test = ed.generate_data_frame_for_specific_engine(ed.df_x_s_test, engine_unit)
        x_test, y_test = ed.generate_x_y_model_inputs(single_engine_test, window=HIST_WINDOW)

        (y_pred, y_true, cm) = mark_001.test_model(x_test, y_test)
        df = ed.df_t.loc[(ed.df_t.unit == engine_unit)]

        print(f"Y_Pred Failure Index: { np.where(y_pred==1)[0][0]} Cycle:{df.iloc[[np.where(y_pred==1)[0][0]]].cycle.values[0]}")
        print(f"Y_True Failure Index: { np.where(y_true==1)[0][0]} Cycle:{df.iloc[[np.where(y_true==1)[0][0]]].cycle.values[0]}")
        
        print(cm.head())

        if GENERATE_PLOTS:
            plot_util.generate_heat_map(cm)
            plot_util.generate_predicted_vs_true_plot(y_true, y_pred, "About To Fail")
            plot_util.generate_about_to_fail_graphs(y_true, y_pred)

In [ ]:
train()

In [ ]:
mark_001.load_model(DATA_PATH+"/Mark_001_Pickle")

In [ ]:
test()